<a href="https://colab.research.google.com/github/LoganToullec/ProgGPU/blob/main/Copie_de_ProgGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook
This notebook is a starting point to setup for CUDA code.
If you want to save your codes, you will have to create a copy of this notebook, and then work on this copy

In [7]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


# Prepare your notebook to handle CUDA code

In [8]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-wr3gmehx
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-wr3gmehx
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 781ff5b76ba6c4c2d80dcbbec9983e147613cc71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.1.0-py3-none-any.whl size=8011 sha256=e2b2acbb79d726227ace2ffdeb18b901e4debb9cf9419b13a35a389a655ee4ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-190ym8mf/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Source files will be saved in "/tmp/tmpw56obh94".


# Install & Use of CCfits for TD 2


In [9]:
!apt-get install libccfits-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libccfits0v5
Suggested packages:
  libccfits-doc
The following NEW packages will be installed:
  libccfits-dev libccfits0v5
0 upgraded, 2 newly installed, 0 to remove and 38 not upgraded.
Need to get 501 kB of archives.
After this operation, 3,105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libccfits0v5 amd64 2.6+dfsg-1 [195 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libccfits-dev amd64 2.6+dfsg-1 [306 kB]
Fetched 501 kB in 0s (1,280 kB/s)
Selecting previously unselected package libccfits0v5:amd64.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../libccfits0v5_2.6+dfsg-1_amd64.deb ...
Unpacking libccfits0v5:amd64 (2.6+dfsg-1) ...
Selecting previously unselected package libccfits-dev:amd64.
Preparing to unpack ...

In [10]:
import astropy.io.fits as pf
import numpy as np

A = np.random.random((10,10))
pf.writeto("test.fits", A, overwrite=True)

In [11]:
%%writefile test2.cu
#include <stdio.h>
#include <CCfits>

int main(void) {
    CCfits::FITS inFile("test.fits", CCfits::Read, true);
    CCfits::PHDU &phdu = inFile.pHDU();
    std::valarray<float> fitsImage;
    phdu.read(fitsImage);
    std::cout << fitsImage[1] << std::endl;
    std::cout << fitsImage.size() << std::endl;
    long naxes[2] = {10,10};
    CCfits::FITS *inFile2 = new CCfits::FITS("test2.fits", FLOAT_IMG, 2, naxes);
    CCfits::PHDU &phdu2 = inFile2->pHDU();
    phdu2.write(1, 99, fitsImage);
}


Writing test2.cu


In [12]:
!nvcc test2.cu -o test2 -I/usr/include/CCfits -L/usr/lib/x86_64-linux-gnu/ -std=c++11 -lCCfits -lcfitsio

In [13]:
!./test2

0.43347
100


In [14]:
import astropy.io.fits as pf
import numpy as np
A = pf.getdata("test2.fits")
B = pf.getdata("test.fits")

# You are ready to code in CUDA

In [36]:
%%cuda
#include <iostream>
#include <fstream>
#include <cstdint>
#include <complex>
#include <chrono>

// Define dimensions of the image for each fractal
const int WIDTH = 10000;
const int HEIGHT = 10000;

// Define the region of the Mandelbrot set
const double M_MIN_X = -2.0;
const double M_MAX_X = 1.0;
const double M_MIN_Y = -1.5;
const double M_MAX_Y = 1.5;
const int M_MAX_ITERATIONS = 1000;

// CUDA kernel to generate fractal
__global__ void generateFractalCUDA(uint8_t* image, double min_x, double max_x, double min_y, double max_y, int max_iterations) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < WIDTH && y < HEIGHT) {
        double cx = min_x + (max_x - min_x) * x / WIDTH;
        double cy = min_y + (max_y - min_y) * y / HEIGHT;
        double zx = 0.0;
        double zy = 0.0;
        int iteration = 0;
        while (zx * zx + zy * zy < 4.0 && iteration < max_iterations) {
            double temp = zx * zx - zy * zy + cx;
            zy = 2.0 * zx * zy + cy;
            zx = temp;
            iteration++;
        }

        // Convert iteration count to color
        uint8_t r, g, b;
        if (iteration == max_iterations) {
            r = g = b = 0; // Black for points inside the set
        } else {
            r = static_cast<uint8_t>(iteration * 255 / max_iterations);
            g = static_cast<uint8_t>((iteration * iteration) % 255);
            b = static_cast<uint8_t>((iteration * iteration * iteration) % 255);
        }

        // Write color to image buffer
        int index = (y * WIDTH + x) * 3;
        image[index] = r;
        image[index + 1] = g;
        image[index + 2] = b;
    }
}

// Function to generate and save a fractal image using CUDA
void generateFractalWithCUDA(const char* filename, double min_x, double max_x, double min_y, double max_y, int max_iterations) {
    uint8_t* h_image = new uint8_t[WIDTH * HEIGHT * 3]; // Host image buffer
    uint8_t* d_image; // Device image buffer

    // Allocate memory on the device
    cudaMalloc((void**)&d_image, WIDTH * HEIGHT * 3 * sizeof(uint8_t));

    // Define block and grid dimensions
    dim3 blockSize(16, 16);
    dim3 gridSize((WIDTH + blockSize.x - 1) / blockSize.x, (HEIGHT + blockSize.y - 1) / blockSize.y);

    auto start = std::chrono::high_resolution_clock::now(); // Start time measurement

    // Launch kernel to generate fractal
    generateFractalCUDA<<<gridSize, blockSize>>>(d_image, min_x, max_x, min_y, max_y, max_iterations);

    // Copy image from device to host
    cudaMemcpy(h_image, d_image, WIDTH * HEIGHT * 3 * sizeof(uint8_t), cudaMemcpyDeviceToHost);

    auto end = std::chrono::high_resolution_clock::now(); // End time measurement
    std::chrono::duration<double> elapsed_seconds = end - start;
    std::cout << "Temps d'execution avec CUDA : " << elapsed_seconds.count() << " secondes." << std::endl;

    // Save image to bitmap file
    std::ofstream outfile(filename, std::ios::binary);
    outfile << "BM";
    int fileSize = WIDTH * HEIGHT * 3 + 54;
    outfile.write(reinterpret_cast<char*>(&fileSize), sizeof(int));
    int reserved = 0;
    outfile.write(reinterpret_cast<char*>(&reserved), sizeof(int));
    int offset = 54;
    outfile.write(reinterpret_cast<char*>(&offset), sizeof(int));
    int headerSize = 40;
    outfile.write(reinterpret_cast<char*>(&headerSize), sizeof(int));
    int width = WIDTH;
    outfile.write(reinterpret_cast<char*>(&width), sizeof(int));
    int height = HEIGHT;
    outfile.write(reinterpret_cast<char*>(&height), sizeof(int));
    short planes = 1;
    outfile.write(reinterpret_cast<char*>(&planes), sizeof(short));
    short bpp = 24;
    outfile.write(reinterpret_cast<char*>(&bpp), sizeof(short));
    int compression = 0;
    outfile.write(reinterpret_cast<char*>(&compression), sizeof(int));
    int imgSize = WIDTH * HEIGHT * 3;
    outfile.write(reinterpret_cast<char*>(&imgSize), sizeof(int));
    int resolutionX = 2835;
    outfile.write(reinterpret_cast<char*>(&resolutionX), sizeof(int));
    int resolutionY = 2835;
    outfile.write(reinterpret_cast<char*>(&resolutionY), sizeof(int));
    int colors = 0;
    outfile.write(reinterpret_cast<char*>(&colors), sizeof(int));
    int importantColors = 0;
    outfile.write(reinterpret_cast<char*>(&importantColors), sizeof(int));

    for (int y = HEIGHT - 1; y >= 0; --y) {
        for (int x = 0; x < WIDTH; ++x) {
            outfile.write(reinterpret_cast<char*>(&h_image[(y * WIDTH + x) * 3 + 2]), sizeof(uint8_t));
            outfile.write(reinterpret_cast<char*>(&h_image[(y * WIDTH + x) * 3 + 1]), sizeof(uint8_t));
            outfile.write(reinterpret_cast<char*>(&h_image[(y * WIDTH + x) * 3]), sizeof(uint8_t));
        }
    }

    // Clean up
    cudaFree(d_image);
    delete[] h_image;

    std::cout << "Image " << filename << " générée avec succès." << std::endl;
}

int main() {
    generateFractalWithCUDA("mandelbrot_with_cuda.bmp", M_MIN_X, M_MAX_X, M_MIN_Y, M_MAX_Y, M_MAX_ITERATIONS);
    // You can add more fractals here with different parameters if needed
    return 0;
}


Temps d'execution avec CUDA : 1.74551 secondes.
Image mandelbrot_with_cuda.bmp générée avec succès.



In [38]:
%%cuda
#include <iostream>
#include <fstream>
#include <cstdint>
#include <complex>

// Définir les dimensions de l'image pour chaque fractale
const int WIDTH = 10000;
const int HEIGHT = 10000;

// Définir la région de l'ensemble de Mandelbrot
const double M_MIN_X = -2.0;
const double M_MAX_X = 1.0;
const double M_MIN_Y = -1.5;
const double M_MAX_Y = 1.5;
const int M_MAX_ITERATIONS = 1000;

// Définir la région de l'ensemble de Julia
const double J_MIN_X = -1.5;
const double J_MAX_X = 1.5;
const double J_MIN_Y = -1.5;
const double J_MAX_Y = 1.5;
const std::complex<double> C(-0.8, 0.156);
const int J_MAX_ITERATIONS = 500;

// Définir la région de l'ensemble de Burning Ship
const double B_MIN_X = -2.0;
const double B_MAX_X = 1.0;
const double B_MIN_Y = -2.0;
const double B_MAX_Y = 2.0;
const int B_MAX_ITERATIONS = 1000;

// Définir la région de l'ensemble de Newton
const double N_MIN_X = -1.0;
const double N_MAX_X = 1.0;
const double N_MIN_Y = -1.0;
const double N_MAX_Y = 1.0;
const int N_MAX_ITERATIONS = 1000;

// Fonction pour évaluer l'ensemble de Mandelbrot
int mandelbrot(double cx, double cy) {
    std::complex<double> c(cx, cy);
    std::complex<double> z(0.0, 0.0);
    int iteration = 0;
    while (std::abs(z) < 2.0 && iteration < M_MAX_ITERATIONS) {
        z = z * z + c;
        iteration++;
    }
    return iteration;
}

// Fonction pour évaluer l'ensemble de Julia
int julia(double x, double y) {
    std::complex<double> z(x, y);
    int iteration = 0;
    while (std::abs(z) < 2.0 && iteration < J_MAX_ITERATIONS) {
        z = z * z + C;
        iteration++;
    }
    return iteration;
}

// Fonction pour évaluer l'ensemble de Burning Ship
int burningShip(double cx, double cy) {
    std::complex<double> c(cx, cy);
    std::complex<double> z(0.0, 0.0);
    int iteration = 0;
    while (std::abs(z) < 2.0 && iteration < B_MAX_ITERATIONS) {
        z = std::complex<double>(std::abs(z.real()), std::abs(z.imag())) * std::complex<double>(std::abs(z.real()), std::abs(z.imag())) + c;
        iteration++;
    }
    return iteration;
}

// Fonction pour évaluer l'ensemble de Newton
int newton(double x, double y) {
    std::complex<double> z(x, y);
    int iteration = 0;
    while (std::abs(z * z * z - 1.0) > 0.001 && iteration < N_MAX_ITERATIONS) {
        z = z - (z * z * z - 1.0) / (3.0 * z * z);
        iteration++;
    }
    return iteration;
}

// Fonction pour convertir une valeur d'itération en une couleur RVB
void iterationToColor(int iteration, int maxIterations, uint8_t& r, uint8_t& g, uint8_t& b) {
    if (iteration == maxIterations) {
        r = g = b = 0; // Noir pour les points à l'intérieur de l'ensemble
    } else {
        // Utiliser une palette de couleurs pour les points à l'extérieur de l'ensemble
        r = static_cast<uint8_t>(iteration * 255 / maxIterations);
        g = static_cast<uint8_t>((iteration * iteration) % 255);
        b = static_cast<uint8_t>((iteration * iteration * iteration) % 255);
    }
}

// Fonction pour générer et enregistrer une image de fractale
void generateFractal(const char* filename, double min_x, double max_x, double min_y, double max_y, int max_iterations, int(*fractalFunction)(double, double)) {
    // Créer un tableau pour stocker les pixels de l'image
    uint8_t image[WIDTH][HEIGHT][3];

    // Parcourir chaque pixel de l'image et calculer la fractale
    for (int x = 0; x < WIDTH; x++) {
        for (int y = 0; y < HEIGHT; y++) {
            // Convertir les coordonnées de l'image en coordonnées du plan complexe
            double cx = min_x + (max_x - min_x) * x / WIDTH;
            double cy = min_y + (max_y - min_y) * y / HEIGHT;
            // Évaluer la fractale pour ces coordonnées
            int iteration = fractalFunction(cx, cy);
            // Convertir la valeur d'itération en couleur RVB
            iterationToColor(iteration, max_iterations, image[x][y][0], image[x][y][1], image[x][y][2]);
        }
    }

    // Enregistrer l'image dans un fichier bitmap
    std::ofstream outfile(filename, std::ios::binary);
    outfile << "BM";
    int fileSize = WIDTH * HEIGHT * 3 + 54;
    outfile.write(reinterpret_cast<char*>(&fileSize), sizeof(int));
    int reserved = 0;
    outfile.write(reinterpret_cast<char*>(&reserved), sizeof(int));
    int offset = 54;
    outfile.write(reinterpret_cast<char*>(&offset), sizeof(int));
    int headerSize = 40;
    outfile.write(reinterpret_cast<char*>(&headerSize), sizeof(int));
    int width = WIDTH;
    outfile.write(reinterpret_cast<char*>(&width), sizeof(int));
    int height = HEIGHT;
    outfile.write(reinterpret_cast<char*>(&height), sizeof(int));
    short planes = 1;
    outfile.write(reinterpret_cast<char*>(&planes), sizeof(short));
    short bpp = 24;
    outfile.write(reinterpret_cast<char*>(&bpp), sizeof(short));
    int compression = 0;
    outfile.write(reinterpret_cast<char*>(&compression), sizeof(int));
    int imgSize = WIDTH * HEIGHT * 3;
    outfile.write(reinterpret_cast<char*>(&imgSize), sizeof(int));
    int resolutionX = 2835;
    outfile.write(reinterpret_cast<char*>(&resolutionX), sizeof(int));
    int resolutionY = 2835;
    outfile.write(reinterpret_cast<char*>(&resolutionY), sizeof(int));
    int colors = 0;
    outfile.write(reinterpret_cast<char*>(&colors), sizeof(int));
    int importantColors = 0;
    outfile.write(reinterpret_cast<char*>(&importantColors), sizeof(int));

    for (int y = HEIGHT - 1; y >= 0; --y) {
        for (int x = 0; x < WIDTH; ++x) {
            outfile.write(reinterpret_cast<char*>(&image[x][y][2]), sizeof(uint8_t));
            outfile.write(reinterpret_cast<char*>(&image[x][y][1]), sizeof(uint8_t));
            outfile.write(reinterpret_cast<char*>(&image[x][y][0]), sizeof(uint8_t));
        }
    }

    outfile.close();
    std::cout << "Image " << filename << " générée avec succès." << std::endl;
}

int main() {
    generateFractal("mandelbrot.bmp", M_MIN_X, M_MAX_X, M_MIN_Y, M_MAX_Y, M_MAX_ITERATIONS, mandelbrot);
    //generateFractal("julia.bmp", J_MIN_X, J_MAX_X, J_MIN_Y, J_MAX_Y, J_MAX_ITERATIONS, julia);
    //generateFractal("burning_ship.bmp", B_MIN_X, B_MAX_X, B_MIN_Y, B_MAX_Y, B_MAX_ITERATIONS, burningShip);
    //generateFractal("newton.bmp", N_MIN_X, N_MAX_X, N_MIN_Y, N_MAX_Y, N_MAX_ITERATIONS, newton);

    return 0;
}
